# Making a Covid-19 choropleth with Python and Folium

While undertaking a geospatial analysis project I had the need to quickly create some choropleth maps overlayed to some other raster and vector layers

From Wikipedia: A choropleth map is a thematic map in which areas are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map, such as population density or per-capita income. Choropleth maps provide an easy way to visualize how a measurement varies across a geographic area or show the level of variability within a region. A heat map is similar but does not use geographic boundaries

Let's start importing libraries we need:

In [ ]:
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import branca as bc
from pyproj import Transformer
import folium
from folium.features import DivIcon
import glob, os
import imageio
import webbrowser
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from isoweek import Week

Let's assign a working directory and create a images folder if one doesn't exist

In [ ]:
dir_path = os.getcwd()
if not os.path.exists('images'):
    os.makedirs('images')
image_path = dir_path + '\\images\\'

To create a choropleth we basically need 2 things:

* a geoJSON file that describes our geographical features, in this example the MSOA boundaries.

* the values to be displayed (as different colours) on the map

The first step is to import a shapefile via geopandas. Geopandas will create a geodataframe and then comvert it to json so that it can be passed to Folium

In [ ]:
# read msoa boundary into dataframe and transform to WGS84
msoa = gpd.read_file(dir_path + "/data/geospatial/MSOAs.shp")
msoa = msoa.to_crs(epsg=4326)
# remove boundaries not in DSFRS service area and convert dataframe to json
msoa = msoa[~msoa["msoa11nm"].str.contains(r'Bath(?!$)') & ~msoa['msoa11nm'].str.contains(r'Wilts(?!$)') & ~msoa['msoa11nm'].str.contains(r'Corn(?!$)') & ~msoa['msoa11nm'].str.contains(r'(?!$)Dors(?!$)') & ~msoa['msoa11nm'].str.contains(r'Corn(?!$)') & ~msoa['msoa11nm'].str.contains(r'Corn(?!$)') & ~msoa['msoa11nm'].str.contains(r'(?!$)Dors(?!$)') & ~msoa['msoa11nm'].str.contains(r'North Somer(?!$)')]
msoa_list = msoa['msoa11cd'].tolist()
msoa_json = msoa.to_json()

The second step is to import the weekly csv file from the .Gov website. We can read it into a pandas dataframe and manipulate to get an output for Folium. In this case we will want a dataframe with each weeks data to generate a gif at the very end.

In [ ]:
df = pd.read_csv(r'https://coronavirus.data.gov.uk/downloads/msoa_data/MSOAs_latest.csv')
# drop redundant columns
df = df.drop(['areaName','newCasesBySpecimenDateRollingRate'], axis = 1)
# rename dataframe columns
df.columns = ['msoa11cd', 'date', 'newCasesBySpecimenDateRollingSum']
# filter only the MSOAs we want (from the boundary layer)
df = df[df['msoa11cd'].isin(msoa_list)]
# remove zeros and -99 values otherwise the choropleth will try and colour them
df['newCasesBySpecimenDateRollingSum'] = df['newCasesBySpecimenDateRollingSum'].replace({0:np.nan, -99:np.nan})
# create a mask to split weekly data from the most recent
mask = df['date'] == df["date"].max()
last_7_days = df[mask]
weekly = df
#weekly = df[~mask]
# create a list of dates from the dataframe
listofweeks = list(last_7_days['date'].map(lambda x: str(x)).unique())
listofweeks.sort()

Lets grab some data to sdd some additional layers like the service area and list of fire stations

In [ ]:
# read station csv into dataframe
df = pd.read_csv((dir_path + "/data/dsfrs_stations.csv"))
# convert x/y columns to number from text
df[["EASTING", "NORTHING"]] = df[["EASTING", "NORTHING"]].apply(pd.to_numeric)

In [ ]:
# create a function to convert x/y from OSGB36 to WGS84
def vectorized_convert(df):
    transformer = Transformer.from_crs("epsg:27700", "epsg:4326")
    converted = transformer.transform(df['EASTING'].tolist(), df['NORTHING'].tolist())
    df['lat'] = converted[1]
    df['lon'] = converted[0]
    return df
vec = vectorized_convert(df)
vec = gpd.GeoDataFrame(vec, geometry=gpd.points_from_xy(vec['lat'], vec['lon']),crs="EPSG:4326")

In [ ]:
# read service boundary into dataframe and transform to WGS84
service_area = gpd.read_file(dir_path + "/data/geospatial/DSFRS_Service_Area.shp")
service_area = service_area.to_crs(epsg=4326)

Add some additional functions to add a custom legend to Folium

In [ ]:
def modify_HeaderLegend(map_folium):    
    """
    """
    # Header to Add
    head = """
    {% macro header(this, kwargs) %}
    <style>
    .leaflet-right {
      float: left !important;
    }
    </style>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css">
    <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
    <script>$( function() {
        $( ".maplegend" ).draggable({
            start: function (event, ui) {
                $(this).css({
                    right: "auto",
                    top: "auto",
                    bottom: "auto"
                });
            }
        });
    });
    </script>
    {% endmacro %}
    """
    
    # Add Header
    macro = bc.element.MacroElement()
    macro._template = bc.element.Template(head)
    map_folium.get_root().add_child(macro)
    
    
    # CSS to Add (on Header)
    css = """
    {% macro header(this, kwargs) %}
    <style type='text/css'>
      .maplegend {
        position: absolute;
        z-index:9999;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        right: 10px;
        bottom: 20px;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 0px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }    
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
      .leaflet-right {
        right: inherit;
        }
    </style>
    {% endmacro %}
    """
    
    # Add CSS (on Header)
    macro = bc.element.MacroElement()
    macro._template = bc.element.Template(css)
    map_folium.get_root().add_child(macro)    

    return map_folium

In [ ]:
def add_CategoricalLegend(map_folium, title, color_by_label):
    """
    """
    body = f"""
    <div id='maplegend {title}' class='maplegend'>
        <div class='legend-title'>{title}</div>
        <div class='legend-scale'>
            <ul class='legend-labels'>"""
    
    # Loop Categories
    for label, color, border_color in color_by_label:
        body += f"""
                <li><span style='background:{color};border: solid 2px {border_color};'></span>{label}</li>"""
    body += """
            <li><span style="background:black;opacity:1; border:2px solid black; margin-left:5px; height:15px; width:15px; border-radius:50%; display:inline-block"></span>DSFRS Station</li>
            </ul>
        </div>
        <!--<div class='legend-source'>Source: <a href="#link to source">Name of source</a></div>-->
    </div>
    """   

    # Add Body
    body = bc.element.Element(body)
    map_folium.get_root().html.add_child(body)
    
    return map_folium

So we need to generate a choropleth for each week within the dataset. We will put our choropleth sript within a loop to automate creation of all the maps. At the same time each map is created. It is exported to html and then a .png is generated from the html and saved for later.

In [ ]:
for w in listofweeks:
    # Build map 
    m = folium.Map(location=(50.779614, -3.42), zoom_start=9.1, zoomSnap=0.1, zoomDelta=0.5, tiles=None, zoom_control=False, control_scale=False, layer_name='map1')
    folium.TileLayer('cartodbpositron',name='Greyscale').add_to(m)

    # create a layer for stations
    layer = folium.FeatureGroup(name='DSFRS Stations', show=True)
    # add marker one by one for each station
    for i in range(0,len(vec)):
       folium.CircleMarker(location=[vec.iloc[i]['lon'], vec.iloc[i]['lat']], radius=2, stroke=False, fill=True, fill_color='Black', fill_opacity=1).add_to(layer)
    layer.add_to(m)

    # color scheme for boundary
    style = {'color': '#a9a9a9', 'opacity': 0.5,  'fillColor': 'transparent'}  # 'lineColor': '#ffffbf' blue
    folium.GeoJson(service_area, name='Service_Area', style_function = lambda x: style).add_to(m)

    # Add format for legend
    label = ['Service Area']
    color = ['transparent']
    border_color = ['dimgrey']
    color_points = zip(label,color,border_color)
    # Add Map Legend
    m = modify_HeaderLegend(m)
    m = add_CategoricalLegend(m, 'Map Legend', color_by_label=color_points)
    
    filtered_weekly = weekly[weekly['date'] == w]
    folium.Choropleth(
         geo_data=msoa_json,
         name='choropleth',
         data=filtered_weekly,
         columns=['msoa11cd', 'newCasesBySpecimenDateRollingSum'],
         key_on='feature.properties.msoa11cd',
         fill_color='Spectral_r',
         fill_opacity=0.7,
         line_opacity=0,
         nan_fill_opacity=0,
         bins = [0,10,25,50,100,150,200,300],
         legend_name='Weekly rolling sum of positive Covid-19 cases'
     ).add_to(m)
    # add marker and label for date to be shown
    ww = 'w/e ' + w
    folium.map.Marker(
         [51.4, -4.5],
         icon=DivIcon(
         icon_size=(0,0),
         icon_anchor=(0,0),
         html=f"""<div style="color: black; font-size: 24pt; white-space: nowrap;">{ww}</div>"""
         )
    ).add_to(m)
    m
    m.save(dir_path + '/images/positive_cases_' + str(w) + '.html')
    tmpurl='file://{path}/{mapfile}'.format(path=dir_path + '\\images',mapfile='positive_cases_' + str(w) + '.html')
    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.get(tmpurl)
    #Give the map tiles some time to load
    time.sleep(6)
    browser.save_screenshot(dir_path + '\\images\\' + 'positive_cases_' + str(w) + '.png')
    browser.quit()
    # Remove html files
    #os.remove('dir_path + '\\images\\' + 'positive_cases_' + str(w) + '.html')

Finally convert our png files to a gif

In [ ]:
os.chdir(image_path)

# Create Gif and remove each .png file
images = list(glob.glob('*.png'))
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))
    #os.remove(file_name)
imageio.mimwrite('SampleMap.gif', image_list, fps=2)